In [82]:
import os
import arcpy
import pandas as pd
from arcgis.gis import GIS
from arcgis.features import FeatureLayer
from arcgis.features import GeoAccessor, GeoSeriesAccessor


In [83]:
gdb = r"Q:\projects\Mwlrs\AEB_Prioritization_Tool\data\AEB_Analysis.gdb"
arcpy.env.workspace= gdb

In [84]:
# create a df of steams sensitivty
fc= 'r20241209_streams_drought_sensitivity_ago'

fields = ["blue_line_key", "gnis_name", "assessment_unit_name","watershed_group_id", "stream_sntvty_summary"]

data = []

rowcount= 0
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        blue_key= row[0]
        name = row[1]
        unit = row[2]
        wgrp = row[3]
        snst = row[4]
        data.append([blue_key, name, unit, wgrp, snst])

        rowcount += 1

df_strm = pd.DataFrame(data, columns=fields)

In [ ]:
# create a df of assesement units
fc= 'r20250117_AEB_Disturbance_Feature_Class'

fields = ["ASSESSMENT_UNIT_SOURCE_ID", "ASSESSMENT_UNIT_NAME", 'WATERSHED_GROUP_ID']

data = []

rowcount= 0
with arcpy.da.SearchCursor(fc, fields) as cursor:
    for row in cursor:
        id= row[0]
        name = row[1]
        wgrp = row[2]
        data.append([id, name, wgrp])

        rowcount += 1

df_asun = pd.DataFrame(data, columns=fields)

In [86]:
# create a df of assesement units/stream intersection (for unnamed Assesment Units)
fcINT= 'r20250117_tempo_assesUnit_stream_intersect'

fieldsNA = ["ASSESSMENT_UNIT_SOURCE_ID", "stream_sntvty_summary"]

dataNA = []

rowcount= 0
with arcpy.da.SearchCursor(fcINT, fieldsNA) as cursor:
    for row in cursor:
        idNA= row[0]
        snstNA = row[1]
        dataNA.append([idNA, snstNA])

        rowcount += 1

df_asun_na = pd.DataFrame(dataNA, columns=fieldsNA)

In [87]:
# Named Assesement units: assign sensitivity levels by matching them with the corresponding named streams.
df_strm = df_strm[df_strm['assessment_unit_name'] != '']
df_asun = df_asun[df_asun['ASSESSMENT_UNIT_NAME'] != 'N/A']

df_strm_gr = df_strm.groupby(["assessment_unit_name", 'watershed_group_id' ,"stream_sntvty_summary"]).size().reset_index(name='count')

# Mapping for ranks
rank_mapping = {
    "Winter & Summer Sensitive": 4,
    "Winter Sensitive": 3,
    "Summer Sensitive": 2,
    "Non-sensitive": 1
}

# Adding the rank column
df_strm_gr["rank"] = df_strm_gr["stream_sntvty_summary"].map(rank_mapping)

df_asun_nm = df_strm_gr.loc[
    df_strm_gr.groupby(["assessment_unit_name", "watershed_group_id"])["rank"].idxmax()
]

df_asun_nm = df_asun_nm.reset_index(drop=True)

# Fix the Watershed ID of Meldrum Creek
df_asun_nm.loc[df_asun_nm['assessment_unit_name'] == 'Meldrum Creek', 'watershed_group_id'] = 215

df_asun_nm.rename(columns={"stream_sntvty_summary": "drought_snstvty"}, inplace=True)
df_asun_nm = df_asun_nm[['assessment_unit_name', 'watershed_group_id','drought_snstvty']]

df_asun_nm = pd.merge(
    df_asun,
    df_asun_nm,
    how= 'left',
    left_on= ['ASSESSMENT_UNIT_NAME', 'WATERSHED_GROUP_ID'],
    right_on= ['assessment_unit_name', 'watershed_group_id']
)

In [88]:
#For the unnamed assessment units: sensitivity levels them the highest sensitivity level of the streams within the assessment unit area
df_asun_na["rank"] = df_asun_na["stream_sntvty_summary"].map(rank_mapping)

df_asun_unn = df_asun_na.loc[df_asun_na.groupby("ASSESSMENT_UNIT_SOURCE_ID")["rank"].idxmax()]

df_asun_unn = df_asun_unn.reset_index(drop=True)

df_asun_unn.rename(columns={"stream_sntvty_summary": "drought_snstvty"}, inplace=True)

In [106]:
# Prepare the final df
df= pd.concat(
    [
    df_asun_nm[['ASSESSMENT_UNIT_SOURCE_ID', 'drought_snstvty']],
    df_asun_unn[['ASSESSMENT_UNIT_SOURCE_ID', 'drought_snstvty']]
    ]
)

df = df.reset_index(drop=True)

df['ASSESSMENT_UNIT_SOURCE_ID'] = df['ASSESSMENT_UNIT_SOURCE_ID'].round().astype(int)

In [ ]:
# populate the FeatureClass with new Drought Sensitivity values
fc= 'r20250117_AEB_Disturbance_Feature_Class'
# Convert dataframe to dictionary for quick lookup
drought_values = df.set_index('ASSESSMENT_UNIT_SOURCE_ID')['drought_snstvty'].to_dict()

# Update the feature class using arcpy UpdateCursor
with arcpy.da.UpdateCursor(fc, ['ASSESSMENT_UNIT_SOURCE_ID', 'Drought_Sensitivity']) as cursor:
    for row in cursor:
        unit_id = row[0]  # ASSESSMENT_UNIT_SOURCE_ID
        if unit_id in drought_values:
            row[1] = drought_values[unit_id]  # Update Drought_Sensitivity
            cursor.updateRow(row)

print("Feature class updated successfully.")

In [ ]:
# populate the AGOL layer with new Drought Sensitivity values
AGO_HOST = 'https://governmentofbc.maps.arcgis.com'
AGO_USERNAME_DSS = 'XXX'
AGO_PASSWORD_DSS = 'XXX'
gis= GIS(AGO_HOST, AGO_USERNAME_DSS, AGO_PASSWORD_DSS, verify_cert=False)

agol_item_id = '7326558fb14c476bbfc8b7b6ddb45ada'

feature_layer = gis.content.get(agol_item_id).layers[0]  # Assuming first layer in the item

# Query all features in the layer
print ('Reading records from AGOL layer')
features = feature_layer.query(where="1=1", out_fields="ASSESSMENT_UNIT_SOURCE_ID, Drought_Sensitivity", return_geometry=False)
features_dict = {f.attributes['ASSESSMENT_UNIT_SOURCE_ID']: f for f in features}

# Update features with new drought sensitivity values
print ('\nUpdating data in batches')
updated_features = []
for index, row in df.iterrows():
    unit_id = row['ASSESSMENT_UNIT_SOURCE_ID']
    if unit_id in features_dict:
        feature = features_dict[unit_id]
        feature.attributes['Drought_Sensitivity'] = row['drought_snstvty']
        updated_features.append(feature)

# Function to batch updates
def batch_update(features, batch_size=100):
    for i in range(0, len(features), batch_size):
        batch = features[i:i + batch_size]
        response = feature_layer.edit_features(updates=batch)
        print(f"..batch {i // batch_size + 1}: Update response:", response['updateResults'][0])

# Batch and update
if updated_features:
    batch_update(updated_features, batch_size=100)  # Adjust batch size as needed
else:
    print("No features updated.")


In [103]:
# get the list of fields and aliases from the Web map (for backup purpose)

from arcgis.mapping import WebMap

webmap_item = gis.content.get("0b56a5d091a04eacb3559cf33a53281f")
webmap = WebMap(webmap_item)

# Recursive function to search for a layer in group layers
def find_layer_by_name(layers, name):
    for layer in layers:
        if layer.get("title") == name:
            return layer
        elif "layers" in layer:  # Check if it's a group layer
            result = find_layer_by_name(layer["layers"], name)
            if result:
                return result
    return None

# Find the specified feature layer by name
layer_name = "Current (2018) Aquatic Ecosystems Disturbance - All"
feature_layer = find_layer_by_name(webmap.layers, layer_name)

if not feature_layer:
    raise ValueError(f"Layer with name '{layer_name}' not found in the Web Map.")

# Check if field aliases are defined in the Web Map
if "popupInfo" in feature_layer:
    fields = feature_layer["popupInfo"]["fieldInfos"]
    data = [{"Field Name": field["fieldName"], "Alias": field.get("label", field["fieldName"])} for field in fields]
else:
    # Fall back to the fields in the Feature Layer URL
    if "url" in feature_layer:
        from arcgis.features import FeatureLayer
        layer_url = feature_layer["url"]
        fl = FeatureLayer(layer_url)
        fields = fl.properties.fields
        data = [{"Field Name": field["name"], "Alias": field.get("alias", field["name"])} for field in fields]
    else:
        raise ValueError(f"The layer '{layer_name}' does not contain aliases in the Web Map or in the Feature Layer.")

# Convert to a DataFrame
df = pd.DataFrame(data)